# run this with 

In [1]:
from __future__ import print_function
from IPython.display import FileLink, FileLinks
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, HBox, VBox, interactive_output
import ipywidgets as widgets


In [2]:
from PIL import Image
from PIL import ImageDraw
import numpy as np

In [3]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

In [35]:
def draw_tree0(d,
               start=0, 
               lam=10, 
               decay=0.8, 
               l0=5, 
               theta=0.5, 
               spread=0.25, 
               spread_decay=0.8, 
               depth=0, 
               max_depth=40, 
               max_n=1000):
    # d.text(xy=start, text=str(depth), fill=(0,0,0,255))
    if depth>max_depth:
        return max_n
    n_children=np.random.poisson(lam=lam/(1+depth))
    if n_children == 0:
        return max_n
    thetas = theta-spread/2+spread*np.random.random(size=n_children)
    for new_theta in thetas:
        end= (start[0]+l0*np.sin(new_theta),start[1]+l0*np.cos(new_theta))
        d.line(xy=(tuple(start),tuple(end)),fill=(0,0,0,255))
        #print(start, end)
        if end[0]>=0 and end[0]<512 and end[1]>=0 and end[1]<512:
            max_n-=draw_tree0(d=d,
                              start=end,
                              lam=lam,
                              decay=decay,
                              l0=l0*decay,
                              theta=new_theta,
                              spread=spread*spread_decay,
                              spread_decay=spread_decay,
                              depth=depth+1,
                              max_depth=max_depth,
                              max_n=max_n)
    return max_n
def make_tree0(start_x=256,
              start_y=256,
              lam=9.5,
              decay=0.75,
              l0=60,
              theta=np.pi,
              spread_decay=0.8,
              spread=np.pi,
              max_depth=5,
              max_n=1000):
  im = Image.new('RGBA', (512,512), (255,255,255,255))
  d=ImageDraw.Draw(im)
  draw_tree0(
      d=d,
      start=np.array([start_x,start_y]),
      lam=lam,
      decay=decay,
      l0=l0,
      theta=theta,
      spread=spread,
      spread_decay=spread_decay,
      depth=0,
      max_depth=max_depth,
      max_n=max_n) 
             
  return im

interact(make_tree0)

interactive(children=(IntSlider(value=256, description='start_x', max=768, min=-256), IntSlider(value=256, des…

<function __main__.make_tree0(start_x=256, start_y=256, lam=9.5, decay=0.75, l0=60, theta=3.141592653589793, spread_decay=0.8, spread=3.141592653589793, max_depth=5, max_n=1000)>

In [41]:
def draw_tree2(d, start, lam, decay, l0, theta_min, depth=0, max_depth=40, max_n=1000):
    # d.text(xy=start, text=str(depth), fill=(0,0,0,255))
    if depth>max_depth:
        return max_n
    n_children=np.random.poisson(lam=lam/(1+depth))
    if n_children == 0:
        return max_n
    # 3d gaussian
    new_branches= np.random.randn(n_children,3)
    # project on sphere (normalize) 
    new_branches/=np.linalg.norm(new_branches, axis=1, keepdims=True)
    # and keep only 2 first dimensions
    new_branches=new_branches[:,:2]
    # keep only upper branches
    new_branches=new_branches[new_branches[0]>np.sin(theta_min)]
    
    for new_branch in new_branches:
        
        end= start+l0*new_branch
        d.line(xy=(tuple(start),tuple(end)),fill=(0,0,0,255))
        if end[0]>=0 and end[0]<512 and end[1]>=0 and end[1]<512:
            max_n-=draw_tree2(
                d=d,
                start=end,
                lam=lam,
                decay=decay,
                l0=l0*decay,
                theta_min=theta_min,
                depth=depth+1,
                max_depth=max_depth,
                max_n=max_n
            )
    return max_n
def make_tree2(start_x=256,
              start_y=256,
              lam=9.5,
              decay=0.75,
              l0=60,
              theta_min=-np.pi,
              max_depth=10,
              max_n=1000):
  im = Image.new('RGBA', (512,512), (255,255,255,255))
  d=ImageDraw.Draw(im)
  draw_tree2(
      d=d,
      start=np.array([start_x,start_y]),
      lam=lam,
      decay=decay,
      l0=l0,
      depth=0,
      max_depth=max_depth,
      max_n=max_n) 
  return im

interact(make_tree2)

interactive(children=(IntSlider(value=256, description='start_x', max=768, min=-256), IntSlider(value=256, des…

<function __main__.make_tree2(start_x=256, start_y=256, lam=9.5, decay=0.75, l0=60, max_depth=10, max_n=1000)>

In [16]:
def draw_tree1(start=[0,0],
              wind=[0,0],
              wind_decay=0.75,
              lambda_=8,
              lambda_decay=0.75,
              radius=50,
              radius_decay=0.75,
              d=None,
              depth=0,
              max_depth=40,
              max_n=1000):
    # d.text(xy=start, text=str(depth), fill=(0,0,0,255))
    if depth>max_depth:
        return max_n
    n_children=np.random.poisson(lam = lambda_)
    if n_children == 0:
        return max_n
    winds = np.array(wind)+radius*np.random.randn(n_children,2)
    for new_wind in winds:
        child = np.array(start)+new_wind 
        d.line(xy=((start[0],start[1]),
                   (child[0],child[1])),
                   fill=(0,0,0,255))
        max_n-=1
        #print(start, end)
        if max_n>=0 and child[0]>=0 and child[0]<512 and child[1]>=0 and child[1]<512:
            max_n=draw_tree1(start=child,
                             wind=wind*wind_decay,
                             wind_decay=wind_decay,
                             lambda_=lambda_*lambda_decay,
                             lambda_decay=lambda_decay,
                             radius=radius*radius_decay,
                             radius_decay=radius_decay,
                             d=d,
                             depth=depth+1,
                             max_depth=max_depth, 
                             max_n=max_n)
    return max_n

def make_tree1(wind_0=25,
              wind_decay=0.75,
              lambda_=8,
              lambda_decay=0.75,
              radius=50,
              radius_decay=0.75,
              max_depth=40,
              max_n=1000):
  im = Image.new('RGBA', (512,512), (255,255,255,255))
  d=ImageDraw.Draw(im)
  draw_tree1(start=np.array([256,256]),
            wind=np.array([wind_0,0]),
            wind_decay=wind_decay,
              lambda_=lambda_,
              lambda_decay=lambda_decay,
              radius=radius,
              radius_decay=radius_decay,
              depth=0,
              max_depth=max_depth,
              max_n=max_n,
              d=d) 
  display(im)

interact(make_tree1)
  

interactive(children=(IntSlider(value=25, description='wind_0', max=75, min=-25), FloatSlider(value=0.75, desc…

<function __main__.make_tree1(wind_0=25, wind_decay=0.75, lambda_=8, lambda_decay=0.75, radius=50, radius_decay=0.75, max_depth=40, max_n=1000)>

In [15]:
interact(make_tree)

interactive(children=(IntSlider(value=25, description='wind_0', max=75, min=-25), FloatSlider(value=0.75, desc…

<function __main__.make_tree(wind_0=25, wind_decay=0.75, lambda_=8, lambda_decay=0.75, radius=50, radius_decay=0.75, max_depth=40, max_n=1000)>

In [8]:
def make_tree(start=(256,256),
          lam=9.5,
          decay=0.75,
          theta_decay=0.75,
          l0=60,
          theta=np.pi,
          spread=np.pi):
  im = Image.new('RGBA', (512,512), (255,255,255,255))
  d=ImageDraw.Draw(im)
  draw_tree(start=start,
            lam=lam,
            decay=decay,
            theta_decay=theta_decay,
            l0=l0,
            theta=theta,
            spread=theta,
            d=d) 
  return im

In [9]:
?draw_tree

Signature:
draw_tree(
    start,
    lam,
    decay,
    l0,
    theta,
    theta_decay,
    spread,
    d,
    depth=0,
    max_depth=40,
    max_n=1000,
)
Docstring: <no docstring>
File:      ~/work/<ipython-input-5-50067805e26c>
Type:      function


In [10]:
def make_and_display_tree(start_x=256, start_y=256, lam=9.5, decay=0.75,
                          theta_decay=0.75, l0=60, theta=np.pi/2, spread=np.pi, dummy_arg=False):
    tree_im=make_tree(start=(start_x,start_y),
                   lam=lam,
                   decay=decay,
                   theta_decay=theta_decay,
                   l0=l0,
                   theta=theta,
                   spread=theta)
    display(tree_im)

In [11]:
interact(make_and_display_tree)

interactive(children=(IntSlider(value=256, description='start_x', max=768, min=-256), IntSlider(value=256, des…

<function __main__.make_and_display_tree(start_x=256, start_y=256, lam=9.5, decay=0.75, theta_decay=0.75, l0=60, theta=1.5707963267948966, spread=3.141592653589793, dummy_arg=False)>

In [29]:
start_x = IntSlider(description=r'\(x_0\)', min=0, max=512, value=256, continuous_update=False)
start_y = IntSlider(description=r'\(y_0\)', min=0, max=512, value=256, continuous_update=False)
lam = FloatSlider(description=r'\(\lambda\)', min=0, max=15, value=9.5, continuous_update=False)
decay = FloatSlider(description=r'\(\delta\)', min=0, max=1.0, value=0.75, continuous_update=False)
theta_decay = FloatSlider(description=r'\(\delta_{\theta}\)', min=0, max=1.0, value=0.75, continuous_update=False)
l0 = IntSlider(description=r'\(l_0\)', min=0,max=100,value=60, continuous_update=False)
theta = FloatSlider(description=r'\(\theta\)', min=0, max=2*np.pi, value=np.pi, continuous_update=False)
spread = FloatSlider(description='spread', min=0, max=2*np.pi, value=np.pi, continuous_update=False)
#toggle_to_redraw = widgets.Checkbox(descritption='new', value=False)
toggle_to_redraw=widgets.ToggleButton(
    description='Redraw',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='redraw a tree',
    icon='check'
)

ui=VBox([
    HBox([start_x, start_y, lam, decay]),
    HBox([decay, theta_decay, theta, l0, spread]),
    toggle_to_redraw
])

out= widgets.interactive_output(f=make_and_display_tree,  
                                controls={'start_x':start_x,
                                 'start_y':start_y,
                                 'lam':lam,
                                 'decay':decay,
                                 'theta':theta,
                                 'theta_decay':theta_decay,
                                 'l0':l0,
                                 'theta':theta,
                                 'spread':spread,
                                 'dummy_arg':toggle_to_redraw
                               }
                                #,layout=widgets.Layout(width='50%', height='80px')
                               )


In [30]:
display(out)
display(ui)


Output(outputs=({'traceback': ['\x1b--------------------------------------------------------------------…

In [28]:
widgets.widget_button.

SyntaxError: invalid syntax (<ipython-input-28-fb4a8a64c9e8>, line 1)

In [19]:
FileLink('./simple interactive.ipynb')

/home/jovyan/work/simple interactive.ipynb

In [23]:
FileLinks('.', recursive=False)

./
  README.md
  simple interactive.ipynb
  .gitignore
  requirements.txt
  LICENSE
  launch.sh